In [2]:
import json 
import os 

with open("files_for_ml/protein_props.json") as f:
    protein_properties = json.load(f)

print("Total Human proteins from Uniprot:", len(protein_properties))

Total Human proteins from Uniprot: 20434


### Flexibility Sequential Properties

In [5]:
protein_properties["P05067"]["Flexibility"]

[0.9811547619047618,
 0.9559285714285715,
 0.9583452380952382,
 0.9518214285714286,
 0.9475238095238097,
 0.960452380952381,
 0.9557261904761906,
 0.9468214285714284,
 0.9840714285714286,
 0.9698095238095238,
 0.9859285714285714,
 0.9796904761904761,
 0.9798452380952383,
 1.0251785714285715,
 0.9729404761904762,
 1.0295595238095239,
 1.0062619047619048,
 1.0345119047619047,
 1.020654761904762,
 1.0251190476190477,
 1.0046309523809522,
 1.0134285714285713,
 0.9820238095238096,
 0.9941904761904762,
 1.0060952380952382,
 1.0219166666666666,
 1.0003214285714284,
 1.0045952380952383,
 0.9791904761904763,
 0.9958333333333333,
 0.9686190476190477,
 0.9566785714285715,
 0.9551547619047618,
 0.9848452380952379,
 0.9673095238095238,
 0.9611309523809526,
 0.997452380952381,
 0.9647619047619047,
 0.9795714285714286,
 0.9784047619047618,
 1.0011785714285715,
 0.9709642857142856,
 1.0211904761904762,
 1.0128452380952382,
 1.0173095238095238,
 1.0400119047619047,
 0.9923333333333333,
 1.0621904761904

In [29]:
import numpy as np
import scipy.stats as stats

def calculate_sequence_properties(seq):
    properties = {}

    # Handle sequence with length 0
    if len(seq) == 0:
        properties = {
            'Mean': 0, 'Mode': 0, 'Min': 0, 'Max': 0, 'Variance': 0,
            'Median': 0, 'Standard Deviation': 0, 'Range': 0,
            'Min gap': 0, 'Max gap': 0, 'Average gap': 0,
            'Min 2-hop gap': 0, 'Max 2-hop gap': 0, 'Average 2-hop gap': 0
        }
        return properties

    # Basic properties
    properties['Mean'] = np.mean(seq)
    properties['Mode'] = stats.mode(seq)[0][0] if len(seq) > 0 else 0
    properties['Min'] = np.min(seq)
    properties['Max'] = np.max(seq)
    properties['Variance'] = np.var(seq)
    properties['Median'] = np.median(seq)
    properties['Standard Deviation'] = np.std(seq)
    properties['Range'] = np.max(seq) - np.min(seq)
    
    # Handle sequence with length 1
    if len(seq) == 1:
        properties['Min gap'] = properties['Max gap'] = properties['Average gap'] = 0
        properties['Min 2-hop gap'] = properties['Max 2-hop gap'] = properties['Average 2-hop gap'] = 0
        return properties

    # Gaps
    gaps = np.diff(seq)
    properties['Min gap'] = np.min(gaps)
    properties['Max gap'] = np.max(gaps)
    properties['Average gap'] = np.mean(gaps)

    # Handle sequence with length 2
    if len(seq) == 2:
        properties['Min 2-hop gap'] = properties['Max 2-hop gap'] = properties['Average 2-hop gap'] = 0
        return properties

    # 2-hop gaps
    two_hop_gaps = [seq[i+2] - seq[i] for i in range(len(seq) - 2)]
    properties['Min 2-hop gap'] = np.min(two_hop_gaps)
    properties['Max 2-hop gap'] = np.max(two_hop_gaps)
    properties['Average 2-hop gap'] = np.mean(two_hop_gaps)

    return properties

In [30]:
flexibility_properties = {}

for protein in protein_properties:
    properties = calculate_sequence_properties(protein_properties[protein]["Flexibility"])
    flexibility_properties[protein] = properties

# Convert to DataFrame
df = pd.DataFrame.from_dict(flexibility_properties, orient='index')

# Rename columns to include '_percent'
df.columns = [col for col in df.columns]
print(len(df.columns))

# Save to CSV
df.to_csv('files_for_ml/flexibility_properties.csv')

# print(df)

/tmp/ipykernel_16227/1884929929.py:19: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  properties['Mode'] = stats.mode(seq)[0][0] if len(seq) > 0 else 0


14
